In [8]:

import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import pickle
import collections

path="data/coqa-train.json"


with open(path,"r")as f:
    train=json.load(f)
    
question_0=collections.Counter()
question_1=collections.Counter()
qa_list=[]
total=0

for paragraph in tqdm(train["data"]):
    for i in range(len(paragraph["questions"])):
        question_dict=paragraph["questions"][i]
        answer_dict=paragraph["answers"][i]
        question_text=question_dict["input_text"]
        question=tokenize(question_text)
        answer_text=answer_dict["input_text"]
        qa_list.append((question_text,answer_text))
        total+=1
        
        if len(question)>0:
            question_0[question[0]]+=1
            if len(question)>1:
                question_1[" ".join(question[0:2])]+=1

        


100%|██████████| 7199/7199 [00:11<00:00, 624.55it/s]


In [18]:
print(*[(q[0],q[1]/total) for q in question_0.most_common()[0:20]],sep="\n")
print(*[(q[0],q[1]/total) for q in question_1.most_common()[0:20]],sep="\n")

('What', 0.23897576555266137)
('Who', 0.11518035472677571)
('How', 0.07971688127605916)
('Did', 0.05545482157813837)
('what', 0.054847349673713956)
('Where', 0.053024933960440696)
('Was', 0.03746997155926993)
('When', 0.033300505306175045)
('who', 0.02869844542417186)
('Is', 0.025835964177565878)
('Why', 0.021307537253674746)
('how', 0.02056200355279023)
('Does', 0.016567415575211464)
('where', 0.013125074783473082)
('And', 0.013106666543945069)
('Which', 0.012683277034800777)
('did', 0.012259887525656484)
('was', 0.009535468075510598)
('when', 0.008210074829493682)
('Were', 0.007667031763417305)
('What did', 0.04188794904599299)
('What was', 0.03865730300882675)
('What is', 0.034883613905584136)
('How many', 0.028689241304407854)
('Who was', 0.025458595267241618)
('Who is', 0.01787440058170037)
('Did he', 0.01754305227019614)
('Where did', 0.014643754544534133)
('What does', 0.013502443693797344)
('When did', 0.01338279013686526)
('Who did', 0.010971310758695592)
('Where was', 0.01005

In [2]:
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in nltk.word_tokenize(sent)]

In [21]:
print(*[(q,a) for q,a in qa_list[0:50]],sep="\n")
print(*[(q.split()[0],a) for q,a in qa_list[0:50]],sep="\n")

('When was the Vat formally opened?', 'It was formally established in 1475')
('what is the library for?', 'research')
('for what subjects?', 'history, and law')
('and?', 'philosophy, science and theology')
('what was started in 2014?', 'a  project')
('how do scholars divide the library?', 'into periods')
('how many?', 'five')
('what is the official name of the Vat?', 'The Vatican Apostolic Library')
('where is it?', 'in Vatican City')
('how many printed books does it contain?', '1.1 million')
('when were the Secret Archives moved from the rest of the library?', 'at the beginning of the 17th century;')
('how many items are in this secret collection?', '150,000')
('Can anyone use this library?', 'anyone who can document their qualifications and research needs.')
('what must be requested to view?', 'unknown')
('what must be requested in person or by mail?', 'Photocopies')
('of what books?', 'only books published between 1801 and 1990')
('What is the Vat the library of?', 'the Holy See')
(

In [28]:
%%bash
head -c 100000 "data/coqa-train.json"

{
  "version": "1.0",
  "data": [
    {
      "source": "wikipedia",
      "id": "3zotghdk5ibi9cex97fepx7jetpso7",
      "filename": "Vatican_Library.txt",
      "story": "The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-y

In [32]:
text=json.dumps(train["data"][0],indent=4)
print(text)

{
    "source": "wikipedia",
    "id": "3zotghdk5ibi9cex97fepx7jetpso7",
    "filename": "Vatican_Library.txt",
    "story": "The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of ma

In [23]:
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import pickle
import collections

def c2wpointer(context_text,context,answer_start,answer_end):#answer_start,endをchara単位からword単位へ変換
    #nltk.tokenizeを使って分割
    #ダブルクオテーションがなぜか変化するので処理
    token_id={}
    cur_id=0
    for i,token in enumerate(context):
        start=context_text.find(token,cur_id)
        token_id[i]=(start,start+len(token))
        cur_id=start+len(token)
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_start and answer_start<=token_id[i][1]:
            answer_start_w=i
            break
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_end and answer_end<=token_id[i][1]:
            answer_end_w=i
            break
    return answer_start_w,answer_end_w

#sentenceを受け取り、tokenizeして返す
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in nltk.word_tokenize(sent)]

def test_process():
    
    dev=pd.read_json("data/coqa-dev.json")
    contexts=[]
    questions=[]
    answer_starts=[]
    answer_ends=[]
    answer_texts=[]
    ids=[]
    word2count=collections.Counter()
    char2count=collections.Counter()

    for i in tqdm(range(len(dev))):

        paragraph=dev.iloc[i][0]
        context_text=paragraph["story"]
        context=tokenize(context_text)

        for word in context:
            word2count[word]+=1
            for char in word:
                char2count[char]+=1

        for i in range(len(paragraph["questions"])):
            question_dict=paragraph["questions"][i]
            answer_dict=paragraph["answers"][i]

            question_text=question_dict["input_text"]
            question=tokenize(question_text)    
            answer_text=answer_dict["input_text"]
    

            for word in question:
                word2count[word]+=1
                for char in word:
                    char2count[char]+=1
                    
            contexts.append(context)
            questions.append(question)
            #answer_start=a["answer_start"]
            #answer_end=a["answer_start"]+len(a["text"])
            #answer_start,answer_end=c2wpointer(context_text,context,answer_start,answer_end)
            #answer_starts.append(answer_start)
            #answer_ends.append(answer_end)
            answer_texts.append(answer_text)
            
    print(len(answer_texts),answer_texts[0:100])

In [24]:
test_process()

100%|██████████| 500/500 [00:02<00:00, 191.75it/s]

7983 ['white', 'in a barn', 'no', 'with her mommy and 5 sisters', 'orange and white', 'no', 'she painted herself', 'the farmer', 'they started laughing', 'a bucket of water', 'licked her face', 'no', 'Asta.', 'a bottle', 'Asta.', 'Yes', 'Yes', 'a note', 'No', "Asta's papa", 'unknown', 'yes', 'unknown', 'An elderly Chinese lady and a little boy', 'Yes', 'a paper carrier bag', 'Yes', 'Nicole', 'Shanghai', 'mother', 'food', 'Yes', 'I am having heart surgery soon, so her mother has decided I need more nutrients', 'an iPad', 'hot soup and a container with rice, vegetables and either chicken, meat or shrimp, sometimes with a kind of pancake', 'I am now working on some more Chinese words', 'Yes', '"Thank you"', 'Yes.', 'Dennis Farina', 'Actor', 'No', 'Yes', 'No', 'Farina was cast in a film', 'Michael Mann', '"Thief"', 'cops or gangsters', 'He joined a TV show cast.', '"Law & Order"', 'Detective Joe Fontana', 'No', 'An expensive car', 'No', 'Flashy', 'No', 'No', 'A cop', 'school', 'No', "go to

In [3]:
from nltk.translate import bleu_score

ref = [["a", "bdd","dddd", "c"]]
hypo = ["a", "dddd", "a"]

print(bleu_score.sentence_bleu(ref, hypo))

0.6474591278836639


/usr/local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [4]:
import json
path="data/squad_train-v1.1.json"

with open(path,"r")as f:
    t=json.load(f)
    contexts=t["contexts"]
    questions=t["questions"]
    answers=t["answers"]
    
print(answers[0:5])

KeyError: 'contexts'

In [ ]:
%%bash
head -c 1000 